In [1]:
import sys, os
import pandas as pd
import xarray as xr

#sys.path.insert(0, '../')
import shnitsel.dynamic

## Initial Conditions

Here we read from the folder ```test_data/sharc/iconds_butene``` the results of four initial conditions of butene.

In the resulting xarray, we can see that we have information available on the energies, forces, NACs, transition and permanent dipoles and the phase of the wavefunctions for all four configurations of butene in three electronic states.

In [12]:
from shnitsel.dynamic import icondparse

iconds_butene = icondparse.dirs_of_iconds(path='./test_data/sharc/iconds_butene')
iconds_butene

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

<xarray.Dataset> Size: 10kB
Dimensions:     (icond: 4, state: 3, state2: 3, direction: 3, statecomb: 3,
                 atom: 12)
Coordinates:
  * statecomb   (statecomb) object 24B MultiIndex
  * from        (statecomb) int64 24B 0 0 1
  * to          (statecomb) int64 24B 1 2 2
  * state       (state) int64 24B 0 1 2
  * state2      (state2) int64 24B 0 1 2
  * atom        (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction   (direction) <U1 12B 'x' 'y' 'z'
    has_forces  (icond) float64 32B nan nan nan nan
  * icond       (icond) int64 32B 0 1 2 3
Data variables:
    energy      (icond, state) float64 96B -156.1 -155.8 ... -155.7 -155.6
    dip_all     (icond, state, state2, direction) float64 864B -0.07191 ... 0...
    dip_perm    (icond, state, direction) float64 288B -0.07191 ... 0.179
    dip_trans   (icond, statecomb, direction) float64 288B -1.02 ... -0.1704
    forces      (icond, state, atom, direction) float64 3kB 0.01074 ... 0.0108
    phases      (icond, state) float64 96B nan nan nan 1.0 ... 1.0 1.0 1.0 1.0
    nacs        (icond, statecomb, atom, direction) float64 3kB -0.01056 ... ...
    atXYZ       (icond, atom, direction) float64 1kB 0.0 0.0 0.0 ... 5.384 1.774
    atNames     (atom) <U1 48B 'C' 'C' 'C' 'C' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'

In [43]:
# save the parsed data to h5netcdf file
savepath = os.path.join(os.getcwd(), 'test_data', 'sharc', 'iconds_butene.hdf5')

iconds_butene.reset_index('statecomb')\
    .to_netcdf(savepath, engine='h5netcdf')


## Trajectories

Here we read from the folder ```test_data/sharc/traj_butene``` the data of two trajectories of butene upon excitation to the S$_1$ state.



In [40]:
from tqdm import tqdm
from shnitsel.dynamic import trajparse

traj_path = os.path.join(os.getcwd(), 'test_data', 'sharc', 'traj_butene')
traj_names = [p for p in os.listdir(traj_path) if p.startswith('TRAJ_0')]

flist = []
for name in tqdm(traj_names):

    id_ = name.split('_')[-1]
    traj = trajparse.read_traj(os.path.join(traj_path, name))
    flist.append(traj.sel(ts=slice(traj.attrs['max_ts'])) # discard uncalculated timesteps
                 .expand_dims(trajid=[id_])
                 .stack(frame=['trajid', 'ts']))

traj_frames_butene = xr.concat(flist, dim='frame').transpose('frame', ...)
traj_frames_butene

  0%|                                                     | 0/2 [00:00<?, ?it/s]

3


 50%|██████████████████████▌                      | 1/2 [00:00<00:00,  2.03it/s]

3


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  2.02it/s]


<xarray.Dataset> Size: 1MB
Dimensions:     (state: 3, frame: 602, state2: 3, direction: 3, statecomb: 3,
                 atom: 12)
Coordinates:
  * statecomb   (statecomb) object 24B MultiIndex
  * from        (statecomb) int64 24B 1 1 2
  * to          (statecomb) int64 24B 2 3 3
  * state       (state) int64 24B 1 2 3
  * state2      (state2) int64 24B 1 2 3
  * atom        (atom) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
  * direction   (direction) <U1 12B 'x' 'y' 'z'
    atNames     (atom) <U1 48B 'C' 'C' 'C' 'C' 'H' 'H' 'H' 'H' 'H' 'H' 'H' 'H'
  * frame       (frame) object 5kB MultiIndex
  * trajid      (frame) object 5kB '00003' '00003' '00003' ... '00002' '00002'
  * ts          (frame) int64 5kB 0 1 2 3 4 5 6 ... 294 295 296 297 298 299 300
Data variables:
    energies    (frame, state) float64 14kB -156.1 -155.7 ... -155.8 -155.7
    dip_all     (frame, state, state2, direction) float64 130kB 0.01909 ... 0...
    dip_perm    (frame, state, direction) float64 43kB 0.01909 ... 0.04969
    dip_trans   (frame, statecomb, direction) float64 43kB -1.099 ... 0.3908
    sdiag       (frame) int64 5kB 2 2 2 2 2 2 2 2 2 2 2 ... 1 1 1 1 1 1 1 1 1 1
    astate      (frame) int64 5kB 2 2 2 2 2 2 2 2 2 2 2 ... 1 1 1 1 1 1 1 1 1 1
    forces      (frame, state, atom, direction) float64 520kB 0.04099 ... 0.0
    has_forces  (frame) bool 602B True True True True ... True True True True
    phases      (frame, state) float64 14kB -1.0 -1.0 -1.0 1.0 ... 1.0 1.0 1.0
    nacs        (frame, statecomb, atom, direction) float64 520kB -0.02625 .....
    atXYZ       (frame, atom, direction) float64 173kB -0.07448 ... -1.232
Attributes:
    max_ts:   300

In [35]:
# data at a certain time-step (ts)
#traj_frames_butene.sel(ts=300)

In [42]:
# save the parsed data to h5netcdf file
savepath = os.path.join(os.getcwd(), 'test_data', 'sharc', 'traj_butene.hdf5')

traj_frames_butene.reset_index('statecomb')\
    .reset_index('frame')\
    .to_netcdf(savepath, engine='h5netcdf')
